In [1]:
import sqlite3
import numpy
dbName = 'movies.db'
inputFile = 'Movies_and_TV_5.json'
con = sqlite3.connect(dbName)
cur = con.cursor()


class myCorr:
    def __init__(self):
        self.count = 0
        self.list1 = []
        self.list2 = []
    def step(self, value1,value2):
        self.list1.append(value1)
        self.list2.append(value2)
    def finalize(self):
        #print(self.list1[:10])
        #print(self.list2[:10])
        if len(self.list1) and len(self.list2):
            self.correlation = numpy.corrcoef(self.list1,self.list2)[0][1]
        else:
            self.correlation = 0.0
        return self.correlation

con.create_aggregate("corr", 2, myCorr)

In [4]:
import convert

convert.convert(dbName,inputFile,("reviewerID","overall","asin")) #these keys may change based on convert.py
create = """
create table if not exists itemInfo (
itemID varchar(25),
occurances int
)
"""

createID = """
insert into itemInfo(itemID) select distinct itemID from itemRatings
"""
occurances = """
UPDATE itemInfo
SET occurances = (select count(*) t from itemRatings where itemID = itemInfo.itemID)
where exists(select count(*) t from itemRatings where itemID = itemInfo.itemID)
"""
cur.execute(create)


# cur.execute("insert into petSupplies (rating) select rating - (select average from (select AVG(rating) as average,username as usern from petSupplies group by username) where usern = petSupplies.username) from petSupplies ") 
# con.commit()

failed to read
load inserted
done


In [5]:
con.commit()
cur.execute(createID)
cur.execute(occurances)
con.commit()

In [2]:

from datetime import datetime

def calcCorrelations(item,threshold):
    small = f"""select itemID1,correlation,occurances from
    (select corr(rating1,rating2) correlation,itemID2,itemID1 from 
    ((select itemID as itemID2,user as user2,rating as rating2  from petSupplies where itemID = "{item}") 
    inner join  (select itemID itemID1,user as user1,rating as rating1 from petSupplies)
    on  user1 = user2) group by itemID1)  
    inner join (select itemID id, occurances from itemInfo where occurances > {threshold}) on itemID1=id order by correlation desc"""
    noOccurances = f"""select itemID1,correlation,occurances from
    (select corr(rating1,rating2) correlation,itemID2,itemID1 from 
    ((select itemID as itemID2,user as user2,rating as rating2  from petSupplies where itemID = "{item}") 
    inner join  (select itemID itemID1,user as user1,rating as rating1 from petSupplies)
    on  user1 = user2) group by itemID1) order by correlation desc"""
    start = datetime.now()
    cur.execute(small)
    result = cur.fetchmany(100)
    print((datetime.now()-start))
    return result
print(calcCorrelations("B002CJIPEK",10))

0:00:00.774326
[('B00025YTZA', 1.0, 44), ('B0007NTBJ6', 1.0, 41), ('B000F1OS20', 1.0, 47), ('B000FS4OYA', 1.0, 194), ('B000OQO69Q', 1.0, 33), ('B002CJIPEK', 1.0, 520), ('B003BYQ09C', 1.0, 44), ('B0040QOYZ2', 1.0, 379), ('B008OV91HU', 1.0, 37), ('B0002DI68I', 0.9999999999999999, 13), ('B0002H3ZI0', 0.9999999999999999, 193), ('B000633UPS', 0.9999999999999999, 13), ('B000668Z96', 0.9999999999999999, 114), ('B0009XJQ0I', 0.9999999999999999, 32), ('B000E8HQ9Q', 0.9999999999999999, 15), ('B000EFL0A0', 0.9999999999999999, 23), ('B000JCWAWA', 0.9999999999999999, 70), ('B000JJFNJK', 0.9999999999999999, 57), ('B000QFT1R2', 0.9999999999999999, 161), ('B00176CGK8', 0.9999999999999999, 118), ('B003ALMW0M', 0.9999999999999999, 273), ('B003JFRQQ4', 0.9999999999999999, 140), ('B0041HUW8S', 0.9999999999999999, 13), ('B004GFN2ZA', 0.9999999999999999, 96), ('B0052SP8JS', 0.9999999999999999, 77), ('B005AP3B8S', 0.9999999999999999, 110), ('B006MUQOBQ', 0.9999999999999999, 156), ('B00008AJH9', 0.99999999999

C:\Users\dtrag\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\dtrag\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\dtrag\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
C:\Users\dtrag\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  